In [109]:
import numpy as np
import glob
from datetime import date

In [111]:
dataset = []
for idx in glob.glob("data/*.csv"):
    if idx == 'data/S&P500.csv': continue
    stock = np.genfromtxt(idx,skip_header=1,usecols=(0,1),delimiter=',',dtype=["S10","f8"])
    data = np.zeros((stock.shape[0],2))
    data[:,0] = np.array([datetime.datetime.strptime(entry[0], "%Y-%m-%d").date().timetuple().tm_yday for entry in stock])
    data[:,1] = np.array([entry[1] for entry in stock])
    dataset.append((idx,data))

In [145]:
num_stocks = 4
data_batches = []

for stock1 in range(0,len(dataset)):
    for stock2 in range(stock1+1,len(dataset)):
        for stock3 in range(stock2+1,len(dataset)):
            for stock4 in range(stock3+1,len(dataset)):
                size = np.min([dataset[stock1][1].shape[0],dataset[stock2][1].shape[0],dataset[stock3][1].shape[0],dataset[stock4][1].shape[0]])
                data_batch = np.hstack((dataset[stock1][1][0:size,:],dataset[stock2][1][0:size,:],dataset[stock3][1][0:size,:],dataset[stock4][1][0:size,:]))
                data_batches.append(data_batch)

In [146]:
def label(current,future):
    label = np.zeros(5) # [stock1, stock2, stock3, stock4, Cash]
    profit = np.true_divide(future - current, current)
    idx = np.argmax(profit)
    if profit[idx] <= 0: idx = -1
    label[idx] = 1
    return label

In [168]:
label_batches = []

for batch in data_batches:
    label_batch = np.zeros((batch.shape[0]-1,5))
    for idx in range(batch.shape[0]-1):
        current, future = batch[idx+1][[1,3,5,7]], batch[idx][[1,3,5,7]]
        label_batch[idx] = label(current,future)
    label_batches.append(label_batch)

In [175]:
X_train = np.vstack((batch[1:,:] for batch in data_batches))

In [176]:
X_train.shape

(264809, 8)

In [177]:
Y_train = np.vstack(np.vstack((label for label in label_batch)) for label_batch in label_batches)

In [178]:
Y_train.shape

(264809, 5)

In [179]:
np.save('X_train',X_train)
np.save('Y_train',Y_train)